## Choosing EU ETS vs Carbon TAX
EU ETS start only at 2005 and Carbon Tax depends on countries that have already implemented it but could go as early as 1990.

In [1]:
import pandas as pd
import statsmodels.api as sm

carbon_pricing_data = pd.read_excel('./ggdp_data/world_bank_carbon_pricing.xlsx', header=1,
                                    sheet_name='Compliance_Price', na_values='-')
carbon_pricing_data = carbon_pricing_data.set_index('Name of the initiative').T
carbon_pricing_data.rename(columns=lambda x: str.lower(x.replace(' ', '_')), inplace=True)
#dropping all metadata rows
carbon_pricing_data = carbon_pricing_data.iloc[7:]
carbon_pricing_data = carbon_pricing_data.apply(pd.to_numeric)
carbon_pricing_data.index = [str(x) for x in carbon_pricing_data.index]

eu_ets = carbon_pricing_data['eu_ets']
finland_carbon_tax = carbon_pricing_data['finland_carbon_tax']
#Finland has two types of carbon tax: Transport fuels, heating fuels which are the same across the whole series
finland_carbon_tax = finland_carbon_tax.iloc[:, 0]

In [2]:
#checking to see if EU ETS can proxy for carbon tax or the other way round. Does any one of them hold and additional information?
ets_joined_carbon_tax = pd.DataFrame(eu_ets).join(finland_carbon_tax)
ets_joined_carbon_tax = ets_joined_carbon_tax.dropna()
ets_joined_carbon_tax.head()

model = sm.OLS.from_formula('eu_ets ~ finland_carbon_tax', data=ets_joined_carbon_tax)
results = model.fit()
results.summary()


C:\Users\yarde\anaconda3\envs\ggdp-research\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 eu_ets   R-squared:                       0.165
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     3.365
Date:                Tue, 07 May 2024   Prob (F-statistic):             0.0842
Time:                        20:38:10   Log-Likelihood:                -86.898
No. Observations:                  19   AIC:                             177.8
Df Residuals:                      17   BIC:                             179.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              7.4432     11.154      0.667      0.514     -16.090      30.976
finland_carbon_tax     0.3522      0.192      1.834      0.084      -0.053       0.757
==============================================================================
Omnibus:                        6.489   Durbin-Watson:                   0.533
Prob(Omnibus):                  0.039   Jarque-Bera (JB):                4.314
Skew:                           1.139   Prob(JB):                        0.116
Kurtosis:                       3.508   Cond. No.                         114.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Analysing Results
I was looking to see if ets and carbon tax are very correlated, if they are then I could probably assume that one of them proxies for the other. We can see that linear relationship between the variables is insignificant. Meaning that they don't proxy linearly for each other. Thus I should probably use one of them or a combination of both when calculating the GGDP results.

## Calculating GGDP
The base formula which I'm going to use is this: $ GGDP = GDP - KtCO_{2} \cdot P_{tCO_2} - Twaste \cdot 74 kWh \cdot P_{1 kWh elec} - GNI \cdot \% NRD/100 $

First term - air pollution
Second Term - waste pollution translated into electricity cost
Third Term - Natural resource depletion

The 74 figure is based on different papers: check this

Managed to find all the data for this: arises the question of what data to include and how.

The problems with this base formula are:
1. doesn't account for defensive costs - the cost of restoring and protecting the environment. Need to find figures for this.
2. the figure 74 isn't really based on anything concrete (the citing is quite unclear)
3. no accounting for green innovation - not even sure if I want to add this, but perhaps I should

The main problem with actually deriving GGDP is data availability and lack of accounting standard that can be implemented using available data.


In [3]:
#Loading Data for ggdp
finland_ggdp = pd.DataFrame()

wdi_data = pd.read_excel('./ggdp_data/P_Data_Extract_From_World_Development_Indicators.xlsx', header=0,
                         sheet_name='Data', na_values='..')
wdi_data = wdi_data.set_index('Country Name').T
wdi_data = wdi_data.drop(['Series Code'])
wdi_data = wdi_data.rename({"Series Name": "variable_name"})
wdi_data = wdi_data.rename(index=lambda x: x[:y] if (y := x.find(' ')) != -1 else x)
wdi_data = wdi_data.dropna(axis='columns', how='all')


def replace_values_from_dict(curr_value, value_dict: dict):
    try:
        return value_dict[curr_value]
    except KeyError:
        return curr_value


wdi_variables_to_replace = {'GNI, PPP (current international $)': 'gni_ppp',
                            'Adjusted savings: carbon dioxide damage (% of GNI)': 'as_co2_damage_gni',
                            'Adjusted savings: natural resources depletion (% of GNI)': 'as_nrd_gni',
                            'Adjusted savings: particulate emission damage (% of GNI)': 'as_ped_gbi',
                            'GNI (current US$)': 'gni_dollar',
                            'CO2 emissions (kt)': 'co2_kt',
                            'Total greenhouse gas emissions (kt of CO2 equivalent)': 'total_gge_kt',
                            'GDP, PPP (current international $)': 'gdp_ppp',
                            'GDP (current US$)': 'gdp_dollar'}

#def lower_only_letters(curr_str:str):


#renaming variables
wdi_data.loc['variable_name'] = (wdi_data.loc['variable_name']).apply(replace_values_from_dict,
                                                                      args=(wdi_variables_to_replace,))


def change_col_names_to_country(df: pd.DataFrame):
    column_name_list = []
    curr_col_name: str
    for curr_col_name, curr_series in df.items():
        #'country-name'_'variable-name'
        new_column_name = f'{curr_col_name.lower()}_{curr_series["variable_name"]}'
        column_name_list.append(new_column_name)
    df.columns = column_name_list


#combine all finland data into one dataframe to avoid problems with dates
finland_data = wdi_data['Finland']
change_col_names_to_country(finland_data)
finland_data = finland_data.drop(index=['Country', 'variable_name'])
finland_data = finland_data.join(finland_carbon_tax, how='left')

finland_data = finland_data.apply(pd.to_numeric)

There's a problem with calculating waste because not enough waste data is available in order to constrcut the waste cost term. So there's a need to do imputation.
I thought it would make sense to do the imputation by regressing waste on gdp, since as the country has more economic output the more waste it's going to have.
We can see that when we run the regression we get a very high R-squared and that the model seems to be $Net Waste = 0.0005 * GDP[PPP] $, the slope coefficient is significant. We can use this result to fill in the missing values all the way to 1990.

In [4]:
#Waste calculation
# Waste generation data preparation and projections
waste_gen_data = pd.read_csv('./ggdp_data/eurostat_generation_of_waste_by_category.csv')
waste_treatment_data = pd.read_csv('./ggdp_data/eurostat_waste_treatment_by_waste_category.csv')

#getting Finland's total waste generation
finland_gen_total = waste_gen_data[(waste_gen_data['geo'] == 'FI') & (waste_gen_data['waste'] == 'TOTAL')]
finland_gen_total.index = finland_gen_total['TIME_PERIOD']
#getting Finland's total energy recovery from waste, since we are pricing waste through energy prices.
finland_energy_recovery = waste_treatment_data[
    (waste_treatment_data['geo'] == 'FI') & (waste_treatment_data['wst_oper'] == 'RCV_E')]
finland_energy_recovery.index = finland_energy_recovery['TIME_PERIOD']

net_waste_data = pd.DataFrame()
#calculating net waste: net_waste = generated_waste - energy_recovered_from_waste
net_waste_data['finland_net_waste'] = finland_gen_total['OBS_VALUE'] - finland_energy_recovery['OBS_VALUE']

net_waste_data.index = [str(x) for x in net_waste_data.index]

#is GDP correlated with waste

gdp_waste_data = net_waste_data.join(finland_data['finland_gdp_ppp'])

model = sm.OLS.from_formula('finland_net_waste ~ finland_gdp_ppp', data=gdp_waste_data)
results = model.fit()
results.summary()

C:\Users\yarde\anaconda3\envs\ggdp-research\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      finland_net_waste   R-squared:                       0.774
Model:                            OLS   Adj. R-squared:                  0.742
Method:                 Least Squares   F-statistic:                     23.97
Date:                Tue, 07 May 2024   Prob (F-statistic):            0.00176
Time:                        20:38:11   Log-Likelihood:                -158.04
No. Observations:                   9   AIC:                             320.1
Df Residuals:                       7   BIC:                             320.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept       -1.773e+07   2.24e+07     -0.792      0.454   -7.07e+07    3.52e+07
finland_gdp_ppp     0.0005   9.76e-05      4.896      0.002       0.000       0.001
==============================================================================
Omnibus:                        0.956   Durbin-Watson:                   2.067
Prob(Omnibus):                  0.620   Jarque-Bera (JB):                0.645
Skew:                           0.244   Prob(JB):                        0.724
Kurtosis:                       1.783   Cond. No.                     1.33e+12
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.33e+12. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [5]:
#adding the model from the regression to the data
finland_data['net_waste'] = finland_data['finland_gdp_ppp'] * results.params['finland_gdp_ppp']

### Electricity Price
We use electricity price to be able to price solid waste. The problem is electricity price is tiered so the calculation has to be done accordingly. What I plan on doing is taking the fractional part of the total waste converted to electricity respective to house-hold use and industrial use in Finland and then look at each one of them as if it were a single consumer. This is an obvious over-simplification but otherwise there is no way of calculating it because of a lack of data.

In [23]:
#joining together pre-2007 data with post-2007 data
from collections import defaultdict


def process_electricity_df(country_code: str, elec_df: pd.DataFrame, with_tax=True) -> pd.DataFrame:
    tax_filter = 'I_TAX' if with_tax else 'X_TAX'
    filtered_df: pd.DataFrame = elec_df[(elec_df['geo'] == country_code) & (elec_df['tax'] == tax_filter)]
    filtered_df['TIME_PERIOD'] = filtered_df['TIME_PERIOD'].apply(lambda x: x[:x.find('-')])
    filtered_df = filtered_df.drop(columns=['OBS_FLAG', 'DATAFLOW', 'LAST UPDATE', 'freq', 'product'])
    annual_data = filtered_df.groupby(['currency', 'TIME_PERIOD'], as_index=False)['OBS_VALUE'].mean()
    return annual_data


def process_electricity_consumption_df(country_code: str, df: pd.DataFrame) -> pd.DataFrame:
    filtered_df: pd.DataFrame = df[(df['geo'] == country_code)]
    filtered_df = filtered_df.drop(columns=['OBS_FLAG', 'DATAFLOW', 'LAST UPDATE', 'freq'])
    annual_data = {}
    for index, row in filtered_df.iterrows():
        current_year = row['TIME_PERIOD']
        if current_year not in annual_data:
            annual_data[current_year] = {'year': current_year, 'hh_consumption': 0, 'nh_consumption': 0}
        current_year_item = annual_data[current_year]
        if row['nrg_bal'] == 'FC_OTH_HH_E':
            current_year_item['year'] = row['TIME_PERIOD']
            current_year_item['hh_consumption'] = row['OBS_VALUE']
            current_year_item['nh_consumption'] -= row['OBS_VALUE']
        elif row['nrg_bal'] == 'FC':
            current_year_item['nh_consumption'] += row['OBS_VALUE']
    #list of dictionaries of years, each item contains year, hh and nh consumption
    annual_data = [annual_data[key] for key in annual_data]
    return pd.DataFrame(annual_data)


elec_hh_pre_2007 = pd.read_csv('./ggdp_data/eurostat_electricity/eurostat_electricity_price_hh_pre_2007.csv', header=0)
elec_hh_pre_2007_processed = process_electricity_df(country_code='FI', elec_df=elec_hh_pre_2007)
elec_hh_post_2007 = pd.read_csv('./ggdp_data/eurostat_electricity/eurostat_electricity_price_hh_post_2007.csv',
                                header=0)
elec_hh_post_2007_processed = process_electricity_df(country_code='FI', elec_df=elec_hh_post_2007)
elec_hh = pd.concat([elec_hh_post_2007_processed, elec_hh_pre_2007_processed])

elec_nh_pre_2007 = pd.read_csv('./ggdp_data/eurostat_electricity/eurostat_electricity_price_nh_pre_2007.csv')
elec_nh_pre_2007_processed = process_electricity_df(country_code='FI', elec_df=elec_nh_pre_2007)
elec_nh_post_2007 = pd.read_csv('./ggdp_data/eurostat_electricity/eurostat_electricity_price_nh_post_2007.csv')
elec_nh_post_2007_processed = process_electricity_df(country_code='FI', elec_df=elec_nh_post_2007)
elec_nh = pd.concat([elec_nh_post_2007_processed, elec_nh_pre_2007_processed])

elec_consumption = pd.read_csv('./ggdp_data/eurostat_electricity/eurostat_electricity_consumption.csv', header=0)
process_electricity_consumption_df(country_code='FI', df=elec_consumption)

print('1234')

# #getting Finland's total waste generation
# finland_gen_total = waste_gen_data[(waste_gen_data['geo'] == 'FI') & (waste_gen_data['waste'] == 'TOTAL')]
# finland_gen_total.index = finland_gen_total['TIME_PERIOD']
# #getting Finland's total energy recovery from waste, since we are pricing waste through energy prices.
# finland_energy_recovery = waste_treatment_data[(waste_treatment_data['geo'] == 'FI') & (waste_treatment_data['wst_oper'] == 'RCV_E')]
# finland_energy_recovery.index = finland_energy_recovery['TIME_PERIOD']

C:\Users\yarde\AppData\Local\Temp\ipykernel_32048\2252222501.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['TIME_PERIOD'] = filtered_df['TIME_PERIOD'].apply(lambda x: x[:x.find('-')])
C:\Users\yarde\AppData\Local\Temp\ipykernel_32048\2252222501.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['TIME_PERIOD'] = filtered_df['TIME_PERIOD'].apply(lambda x: x[:x.find('-')])
C:\Users\yarde\AppData\Local\Temp\ipykernel_32048\2252222501.py:8: SettingWithCopyWarning: 
A value is tryin

KeyboardInterrupt: 

In [ ]:
#calculating GGDP with current data
#Calculating GGDP using base formula

#emission cost
#carbon tax is us$/ton of CO2
finland_ggdp['co2_emission_cost'] = finland_data['finland_co2_kt'] * 1000 * finland_data['finland_carbon_tax']

#waste cost
#need to add electricity price
finland_ggdp['waste_cost'] = finland_data['net_waste'] * 74 * finland_data['electricity_price']


#natural resource depletion
#finland_ggdp['nrd_cost'] = finland_data['finland_gni_ppp'] * (finland_data['as_nrd_gni']/100)


#finland_data.head()
# finland_ggdp['Finland','env_cost'] = 123 
# finland_ggdp['Finland', 'depletion_cost'] = 123
# finland_ggdp['Finland'] = finland_data['GDP'] - finland_data['env_cost'] - finland_data['']
# 
# 
# 
# finland_data.head()
# 
# #carbon_pricing_data.rename(columns = lambda x: str.lower(x.replace(' ', '_')), inplace=True)
# #dropping all metadata rows
# #carbon_pricing_data = carbon_pricing_data.iloc[7:]
# #carbon_pricing_data = carbon_pricing_data.apply(pd.to_numeric)
# 
# #eu_ets = carbon_pricing_data['eu_ets']
# #finland_carbon_tax = carbon_pricing_data['finland_carbon_tax']
# #Finland has two types of carbon tax: Transport fuels, heating fuels which are the same across the whole series
# #finland_carbon_tax = finland_carbon_tax.iloc[:,0]
